In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

import datetime
import math
import gc

%matplotlib inline

In [2]:
path = "./data/"
train_data = pd.read_table(path + 'train.txt',sep=" ")

In [3]:
test_data = pd.read_table(path + 'test.txt',sep=" ")
test_data['is_trade'] = 0
train_data = pd.concat([train_data,test_data],ignore_index=True)

去掉重复样本，与test数据集保持一致

In [5]:
train_data = train_data.drop_duplicates()

In [6]:
train_data["datetime"] = train_data["context_timestamp"].apply(lambda x: datetime.datetime.fromtimestamp(x))

In [7]:
train_data["day"] = train_data["datetime"].apply(lambda x: x.day)
train_data["hour"] = train_data["datetime"].apply(lambda x: x.hour)

按时间排序

In [8]:
train_data = train_data.sort_values('context_timestamp')

In [9]:
train_data["item_category_list"] = train_data["item_category_list"].apply(lambda x: x.split(";"))
train_data["item_property_list"] = train_data["item_property_list"].apply(lambda x: x.split(";"))

In [10]:
categories = train_data["predict_category_property"].apply(lambda x: x.split(";"))
train_data["num_query_cat"] = categories.apply(lambda x: len(x))
for i in range(categories.apply(lambda x: len(x)).max()):
    train_data["category_"+str(i)] = categories.apply(lambda x: x[i].split(":")[0] if len(x)>i else "-1")
    train_data["category_"+str(i)+"_props"] = categories.apply(lambda x: x[i].split(":")[1].split(",") if len(x)>i and x[i].split(":")[0] != "-1" else ["-1"])

In [12]:
def get_query_corr(df):
    data = df[[ 'category_0',
       'category_0_props', 'category_1', 'category_1_props', 'category_2',
       'category_2_props', 'category_3', 'category_3_props', 'category_4',
       'category_4_props', 'category_5', 'category_5_props', 'category_6',
       'category_6_props', 'category_7', 'category_7_props', 'category_8',
       'category_8_props', 'category_9', 'category_9_props', 'category_10',
       'category_10_props', 'category_11', 'category_11_props', 'category_12',
       'category_12_props', 'category_13', 'category_13_props','item_property_list','item_category_list',
        'num_query_cat','cat_prop_corr','cat_corr']].values
    
    for list in data:
        count1 = 0
        count2 = 0
        for num in range(list[-3]):
            if list[num*2] != "-1" and list[num*2] in list[-4]:
                count1 = count1 + 1 + list[-4].index(list[num*2])
                for prop in list[num*2+1]:
                    if prop != "-1" and prop in list[-5]:
                        count2 = count2 + 1
        list[-1] = count1
        list[-2] = count2

In [13]:
train_data["cat_corr"] = 0
train_data["cat_prop_corr"] = 0
get_query_corr(train_data)

In [14]:
train_data["num_item_category"] = train_data["item_category_list"].apply(lambda x: len(x)-x.count("-1"))
train_data["num_item_property"] = train_data["item_property_list"].apply(lambda x: len(x)-x.count("-1"))

统计各类别在此次出现前的count数

In [15]:
def count_cat_prep(df,column,newcolumn):
    count_dict = {}
    df[newcolumn] = 0
    data = df[[column,newcolumn]].values
    for cat_list in data:
        if cat_list[0] not in count_dict:
            count_dict[cat_list[0]] = 0
            cat_list[1] = 0
        else:
            count_dict[cat_list[0]] += 1
            cat_list[1] = count_dict[cat_list[0]]
    df[[column,newcolumn]] = data

In [16]:
train_data['user_item_id'] = train_data['user_id'].astype(str)+"_"+train_data['item_id'].astype(str)

In [17]:
train_data['user_shop_id'] = train_data['user_id'].astype(str)+"_"+train_data['shop_id'].astype(str)

In [18]:
train_data['user_brand_id'] = train_data['user_id'].astype(str)+"_"+train_data['item_brand_id'].astype(str)

In [20]:
train_data['item_category'] = train_data['item_category_list'].astype(str)

In [21]:
train_data['user_category_id'] = train_data['user_id'].astype(str)+"_"+train_data['item_category'].astype(str)

统计各类别在总样本中的count数

In [ ]:
for column in ['user_id','item_id','item_brand_id','shop_id','user_item_id','user_shop_id','user_brand_id','user_category_id']:
    count_cat_prep(train_data,column,column+'_click_count_prep')

In [ ]:
for column in ['user_id','item_id','item_brand_id','shop_id','user_item_id','user_shop_id','user_brand_id','user_category_id']:
    train_data = train_data.join(train_data[column].value_counts(),on = column ,rsuffix = '_count')

前一次或后一次点击与现在的时间差（trick）

In [ ]:
def lasttime_delta(column):    
    train_data[column+'_lasttime_delta'] = 0
    data = train_data[['context_timestamp',column,column+'_lasttime_delta']].values
    lasttime_dict = {}
    for df_list in data:
        if df_list[1] not in lasttime_dict:
            df_list[2] = -1
            lasttime_dict[df_list[1]] = df_list[0]
        else:
            df_list[2] = df_list[0] - lasttime_dict[df_list[1]]
            lasttime_dict[df_list[1]] = df_list[0]
    train_data[['context_timestamp',column,column+'_lasttime_delta']] = data

In [ ]:
def nexttime_delta(column):    
    train_data[column+'_nexttime_delta'] = 0
    data = train_data[['context_timestamp',column,column+'_nexttime_delta']].values
    nexttime_dict = {}
    for df_list in data:
        if df_list[1] not in nexttime_dict:
            df_list[2] = -1
            nexttime_dict[df_list[1]] = df_list[0]
        else:
            df_list[2] = nexttime_dict[df_list[1]] - df_list[0]
            nexttime_dict[df_list[1]] = df_list[0]
    train_data[['context_timestamp',column,column+'_nexttime_delta']]= data

In [ ]:
for column in ['user_id','item_id','item_brand_id','shop_id','user_item_id','user_shop_id','user_brand_id','user_category_id']:
    lasttime_delta(column)

In [ ]:
train_data = train_data.sort_values('context_timestamp',ascending=False)

In [ ]:
for column in ['user_id','item_id','item_brand_id','shop_id','user_item_id','user_shop_id','user_brand_id','user_category_id']:
    nexttime_delta(column)

In [ ]:
train_data = train_data.sort_values('context_timestamp')

对各类别ctr转化率做贝叶斯平滑
18用19,20号数据进行预估
19用18号数据预估
20号用18,19号数据预估
21号用18,19,20号数据预估
以此类推

In [ ]:
from ctr_PH import ctr_PH

In [ ]:
for column in ['item_city_id','item_id','item_brand_id','user_id','shop_id']:
    train_data[column+'_PH_ctr'] = -1

In [ ]:
df_train = train_data[np.logical_and(train_data.day>18,train_data.day<=20)]
dic_PH = ctr_PH(df_train,train_data[train_data.day==18],'item_city_id',50000,0.0000001)
train_data.loc[train_data.day==18,'item_city_id'+'_PH_ctr'] = train_data.loc[train_data.day==18,'item_city_id'].apply(lambda x: dic_PH[x])
        
dic_PH = ctr_PH(df_train,train_data[train_data.day==18],'item_id',1000,0.002)
train_data.loc[train_data.day==18,'item_id'+'_PH_ctr'] = train_data.loc[train_data.day==18,'item_id'].apply(lambda x: dic_PH[x])
        
dic_PH = ctr_PH(df_train,train_data[train_data.day==18],'item_brand_id',1000,0.002)
train_data.loc[train_data.day==18,'item_brand_id'+'_PH_ctr'] = train_data.loc[train_data.day==18,'item_brand_id'].apply(lambda x: dic_PH[x])
        
dic_PH = ctr_PH(df_train,train_data[train_data.day==18],'user_id',50,0.002)
train_data.loc[train_data.day==18,'user_id'+'_PH_ctr'] = train_data.loc[train_data.day==18,'user_id'].apply(lambda x: dic_PH[x])
        
dic_PH = ctr_PH(df_train,train_data[train_data.day==18],'shop_id',1000,0.002)
train_data.loc[train_data.day==18,'shop_id'+'_PH_ctr'] = train_data.loc[train_data.day==18,'shop_id'].apply(lambda x: dic_PH[x])

In [ ]:
for day in train_data.day.unique():
    if day != 18:
        df_train = train_data[train_data.day<day]
        dic_PH = ctr_PH(df_train,train_data[train_data.day==day],'item_city_id',50000,0.0000001)
        train_data.loc[train_data.day==day,'item_city_id'+'_PH_ctr'] = train_data.loc[train_data.day==day,'item_city_id'].apply(lambda x: dic_PH[x])
        
        dic_PH = ctr_PH(df_train,train_data[train_data.day==day],'item_id',1000,0.002)
        train_data.loc[train_data.day==day,'item_id'+'_PH_ctr'] = train_data.loc[train_data.day==day,'item_id'].apply(lambda x: dic_PH[x])
        
        dic_PH = ctr_PH(df_train,train_data[train_data.day==day],'item_brand_id',1000,0.002)
        train_data.loc[train_data.day==day,'item_brand_id'+'_PH_ctr'] = train_data.loc[train_data.day==day,'item_brand_id'].apply(lambda x: dic_PH[x])
        
        #dic_PH = ctr_PH(df_train,train_data[train_data.day==day],'user_id',50,0.002)
        #train_data.loc[train_data.day==day,'user_id'+'_PH_ctr'] = train_data.loc[train_data.day==day,'user_id'].apply(lambda x: dic_PH[x])
        
        dic_PH = ctr_PH(df_train,train_data[train_data.day==day],'shop_id',1000,0.002)
        train_data.loc[train_data.day==day,'shop_id'+'_PH_ctr'] = train_data.loc[train_data.day==day,'shop_id'].apply(lambda x: dic_PH[x])

In [ ]:
df_train = train_data[train_data.day<25]
dic_PH = ctr_PH(df_train,train_data[train_data.day==25],'item_city_id',50000,0.0000001)
train_data.loc[train_data.day==25,'item_city_id'+'_PH_ctr'] = train_data.loc[train_data.day==25,'item_city_id'].apply(lambda x: dic_PH[x])
        
dic_PH = ctr_PH(df_train,train_data[train_data.day==25],'item_id',1000,0.002)
train_data.loc[train_data.day==25,'item_id'+'_PH_ctr'] = train_data.loc[train_data.day==25,'item_id'].apply(lambda x: dic_PH[x])
        
dic_PH = ctr_PH(df_train,train_data[train_data.day==25],'item_brand_id',1000,0.002)
train_data.loc[train_data.day==25,'item_brand_id'+'_PH_ctr'] = train_data.loc[train_data.day==25,'item_brand_id'].apply(lambda x: dic_PH[x])
        
dic_PH = ctr_PH(df_train,train_data[train_data.day==25],'user_id',50,0.002)
train_data.loc[train_data.day==25,'user_id'+'_PH_ctr'] = train_data.loc[train_data.day==25,'user_id'].apply(lambda x: dic_PH[x])
        
dic_PH = ctr_PH(df_train,train_data[train_data.day==25],'shop_id',1000,0.002)
train_data.loc[train_data.day==25,'shop_id'+'_PH_ctr'] = train_data.loc[train_data.day==25,'shop_id'].apply(lambda x: dic_PH[x])

In [ ]:
统计各类别在此次出现前的trade转化数

In [14]:
def trade_prep_count(column):
    train_data[column+'_trade_prep_count'] = 0
    for day in train_data.day.unique():
        if day == 18:
            train_data.loc[train_data.day==day,column+'_trade_prep_count'] = -1
        else:
            trade_dict = train_data[train_data.day<day].groupby(column)['is_trade'].sum().to_dict()
            train_data.loc[train_data.day==day,column+'_trade_prep_count'] = train_data.loc[train_data.day==day,column].apply(lambda x: trade_dict[x] if x in trade_dict else 0)

In [37]:
for column in ['user_id','item_id','item_brand_id','shop_id','user_item_id','user_shop_id','user_brand_id','user_category_id']:
    trade_prep_count(column)

In [ ]:
for i in ['item_price_level', 'item_sales_level',
                 'item_pv_level', 'user_gender_id', 'user_occupation_id',
                'user_age_level', 
                'context_page_id', 'hour', 'shop_review_num_level',
            "shop_star_level",  "user_star_level" , "item_collected_level",
        'item_category']:
    df_train = train_data[train_data.day<25]
    train_data[i+'_PH_ctr'] = 0
    dic_PH = ctr_PH(df_train,train_data,i,10000,0.00001)
    train_data[i+'_PH_ctr'] = train_data[i].apply(lambda x: dic_PH[x])

对特征进行强行交叉后，利用xgboost进行特征选择

In [11]:
item_prop_list = [
 'item_price_level',
 'item_sales_level',
 'item_pv_level', 'item_collected_level','item_category']

user_prop_list = [ 'user_gender_id',
 'user_occupation_id',
 'user_age_level','user_star_level']


shop_prop_list = ['shop_review_num_level',
 'shop_star_level']
 
other_prop_list = ['context_page_id',
 'hour']
id_prop_list = ['item_brand_id','item_city_id']

In [12]:
cross_list = []
ctr_list = []
for item_prop in item_prop_list:
    for user_prop in user_prop_list:
        train_data[item_prop+'_'+user_prop] = train_data[item_prop].astype(str)+train_data[user_prop].astype(str)
        cross_list.append(item_prop+'_'+user_prop)
        ctr_list.append(item_prop+'_'+user_prop)

In [13]:
for other_prop in other_prop_list:
    for prop in user_prop_list:
        train_data[prop+'_'+other_prop] = train_data[prop].astype(str)+train_data[other_prop].astype(str)
        cross_list.append(prop+'_'+other_prop)
        ctr_list.append(prop+'_'+other_prop)

In [14]:
for user_prop in user_prop_list:
    for shop_prop in shop_prop_list:
        train_data[user_prop+'_'+shop_prop] = train_data[user_prop].astype(str)+train_data[shop_prop].astype(str)
        cross_list.append(user_prop+'_'+shop_prop)
        ctr_list.append(user_prop+'_'+shop_prop)

In [15]:
for user_prop in user_prop_list:
    for id_prop in id_prop_list:
        train_data[user_prop+'_'+id_prop] = train_data[user_prop].astype(str)+'_'+train_data[id_prop].astype(str)
        cross_list.append(user_prop+'_'+id_prop)

In [21]:
for column in cross_list:
    count_cat_prep(train_data,column,column+'_click_count_prep')
    trade_prep_count(column)
    train_data = train_data.join(train_data[column].value_counts(),on = column ,rsuffix = '_count')

In [23]:
for column in cross_list:
    lasttime_delta(column)
train_data = train_data.sort_values('context_timestamp',ascending=False)
for column in cross_list:
    nexttime_delta(column)
train_data = train_data.sort_values('context_timestamp')

In [ ]:
gc.collect()

In [76]:
features = ['context_page_id_PH_ctr', 'hour', 'hour_PH_ctr', 'item_brand_id',
       'item_brand_id_PH_ctr', 'item_brand_id_click_count_prep',
       'item_brand_id_nexttime_delta', 'item_brand_id_trade_prep_count',
       'item_category_PH_ctr', 'item_category_user_age_level_nexttime_delta',
       'item_city_id', 'item_city_id_PH_ctr', 'item_collected_level',
       'item_id', 'item_id_PH_ctr', 'item_id_click_count_prep',
       'item_id_count', 'item_id_lasttime_delta', 'item_id_nexttime_delta',
       'item_id_trade_prep_count', 'item_price_level',
       'item_price_level_PH_ctr',
       'item_price_level_user_star_level_nexttime_delta',
       'item_pv_level_user_age_level_lasttime_delta',
       'item_pv_level_user_occupation_id_click_count_prep',
       'item_pv_level_user_occupation_id_lasttime_delta', 'item_sales_level',
       'item_sales_level_PH_ctr',
       'item_sales_level_user_gender_id_lasttime_delta',
       'item_sales_level_user_star_level_nexttime_delta', 'num_item_property',
       'shop_id', 'shop_id_PH_ctr', 'shop_id_click_count_prep',
       'shop_id_count', 'shop_id_lasttime_delta', 'shop_id_nexttime_delta',
       'shop_review_num_level_PH_ctr', 'shop_review_positive_rate',
       'shop_score_delivery', 'shop_score_description', 'shop_score_service',
       'shop_star_level', 'shop_star_level_PH_ctr', 'user_age_level',
       'user_age_level_PH_ctr',
       'user_age_level_shop_review_num_level_nexttime_delta',
       'user_brand_id_lasttime_delta', 'user_category_id_click_count_prep',
       'user_category_id_lasttime_delta', 'user_category_id_nexttime_delta',
       'user_gender_id_item_brand_id_nexttime_delta',
       'user_gender_id_item_brand_id_trade_prep_count', 'user_id_count',
       'user_id_lasttime_delta', 'user_id_nexttime_delta',
       'user_item_id_click_count_prep', 'user_item_id_lasttime_delta',
       'user_item_id_nexttime_delta',
       'user_occupation_id_item_city_id_click_count_prep',
       'user_occupation_id_item_city_id_lasttime_delta',
       'user_shop_id_lasttime_delta', 'user_shop_id_nexttime_delta',
       'user_star_level', 'user_star_level_PH_ctr',
       'user_star_level_hour_count',
       'user_star_level_item_brand_id_nexttime_delta',
       'user_star_level_item_city_id_lasttime_delta',
       'user_star_level_item_city_id_nexttime_delta',
       'user_star_level_item_city_id_trade_prep_count',
       'user_star_level_shop_star_level_click_count_prep',
       'user_star_level_shop_star_level_nexttime_delta']

In [49]:
train_data[features+['day','is_trade','instance_id']].to_csv(path+'all_feat_0.csv')

In [9]:
path = './data/'
train_data = pd.read_csv(path+'all_feat_0.csv')

In [10]:
train_data = train_data.set_index('Unnamed: 0')

In [ ]:
train_data.loc[train_data.day <= 24,['is_trade']+features].to_csv(path+'baseline_5_train.txt',index=False)
train_data.loc[train_data.day == 25,['is_trade']+features].to_csv(path+'baseline_5_test.txt',index=False)